In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# pd.set_option('display.max_columns', 10)
# pd.set_option('display.max_rows', 100)


In [ ]:
ds_path = r".\data\GermanyUsedCars.zip"
dfo = pd.read_csv(ds_path, index_col=0, dtype_backend='pyarrow')
dfo.columns

### Data exploration

In [ ]:
# dfo.shape
# dfo.isnull().count()
# dfo.isna().sum()
# dfo.isnull().sum()
# dfo.fuel_type.value_counts()
# dfo.color.value_counts()
# dfo.color.isna().sum()
# df.dtypes
# dfo.mileage_in_km.value_counts()
# dfo.mileage_in_km.isna().sum()
# dfo.dtypes




### Data Cleansing

In [ ]:
# drop non-numeric year values
df = dfo.copy()
df.year = pd.to_numeric(df.year, errors='coerce').dropna(axis=0)
df = df.loc[~((df.year.isna()) | (df.year > 2023))]
        # assign(years_in_service = lambda x: 2023-x.year)
# # drop non-numeric price_in_euro values
# df = df.assign(price_in_euro = pd.to_numeric(df.price_in_euro, errors='coerce'))
# df.year = df.year.fillna(0)
        
"number_of_droped_col:", dfo.shape[0]- df.shape[0]

In [ ]:
# df.year.value_counts() - 
df.year.value_counts()

In [ ]:
def check_non_numeric(s:pd.Series):
    
    idx = pd.to_numeric(s, errors='coerce').isna()
    return idx  # return index of non numeric


In [ ]:
# df['year_inroduced'].value_counts().head().plot(kind='bar'| 'barh' | 'hist' , title=, xlabel=)
df['year'].value_counts().plot(kind='barh')
plt.show()

### Working 

In [ ]:
# dfo.head()
# dfo.brand.value_counts()
# dfo[['year', 'registration_date']].head()
# df = dfo.assign(year_e = pd.to_numeric(dfo.year, errors='coerce'))
# df = dfo.assign(years_in_service = 2023-pd.to_numeric(dfo.year, errors='coerce'))
# dfo.year.astype(int, errors='ignore') - 2023
df[['brand', 'year', 'price_in_euro']].groupby(['brand', 'year']).mean()
# 2023 - dfo.year.astype(int, errors='ignore')
# df.head()
# df.years_on_service
# df.years_in_service = 2023 - df.years_in_service
# df = df.loc[df.years_in_service >= 0]
# df[['year', 'year_e']].value_counts()
# df[['year']].value_counts()


In [ ]:
# pd.to_numeric(dfo.year, errors='coerce').value_counts()
pd.to_numeric(df.price_in_euro, errors='raise').value_counts().sort_index()
